In [26]:
! pip install -U spacy -q 

In [27]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.2                         
Location         c:\Users\kingu\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy
Platform         Windows-11-10.0.26100-SP0     
Python version   3.12.7                        
Pipelines        pt_core_news_lg (3.8.0)       



In [28]:
# !wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("pt")
db = DocBin()

In [2]:
import json
with open('training_data.json', encoding='utf-8') as f:
    TRAIN_DATA = json.load(f)

In [3]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy")

100%|██████████| 500/500 [00:00<00:00, 1478.43it/s]


In [4]:
! python -m spacy init config config.cfg --lang pt --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: pt
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [5]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     41.14    0.00    0.00    0.00    0.00
  0     200       7940.66   2880.02   41.83   50.05   35.93    0.42
  0     400        143.94    787.28   52.00   63.83   43.86    0.52
  1     600         54.28    710.89   53.10   60.27   47.46    0.53
  1     800         69.52    813.23   58.20   65.09   52.62    0.58
  2    1000        511.78    815.26   59.44   67.79   52.92    0.59
  2    1200        195.87    844.52   60.75   66.94   55.61    0.61
  3    1400        137.10    907.28   69.14   69.19   69.09    0.69
  4    1600        164.63    927.75 

In [6]:
nlp_ner = spacy.load("model-best") 

In [7]:
doc = nlp_ner('''Meu tablet, que foi dado pelo Governo do Estado do Ceará, não desativa mais o modo de rotação de tela, já tentei de todos os jeitos. Queria saber um modo de resolver isso.''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)